<a href="https://colab.research.google.com/github/email2manoraj/hello-world/blob/master/RAG/LLM_Openrouter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://openrouter.ai/settings/keys

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.4 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 90.8 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.4 MB/s eta 0:00:00


In [5]:
!git clone https://github.com/deepanrajm/deep_learning.git

Cloning into 'deep_learning'...
remote: Enumerating objects: 2993, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (283/283), done.
remote: Total 2993 (delta 95), reused 259 (delta 42), pack-reused 2667 (from 2)
Receiving objects: 100% (2993/2993), 308.70 MiB | 37.20 MiB/s, done.
Resolving deltas: 100% (219/219), done.
Updating files: 100% (2618/2618), done.


In [11]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"  # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="deepseek/deepseek-r1-distill-llama-70b:free",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=False
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm([HumanMessage(content=query)])
    print("\nAssistant:", response.content)



You: How many R in "STRAWBERRRY"

Assistant: To determine how many times the letter **R** appears in the word **STRAWBERRRY**, follow these steps:

1. **Write Down the Word:**
   
   ```
   S T R A W B E R R R Y
   ```

2. **Identify and Count Each 'R':**
   
   - **1st R:** The third letter.
   - **2nd R:** The eighth letter.
   - **3rd R:** The ninth letter.
   - **4th R:** The tenth letter.

3. **Total Count:**
   
   There are **4** occurrences of the letter **R**.

**Final Answer:**
\[
\boxed{4}
\]

You: exit
Goodbye!


In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader("/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])


In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from copy import deepcopy

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader(r"/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
semantic_chunks = text_splitter.split_documents(docs)

# manually apply overlap across those semantic chunks
def apply_overlap_to_chunks(chunks, overlap_characters=200):
    overlapped_chunks = []

    for i in range(len(chunks)):
        curr = deepcopy(chunks[i])
        if i > 0:
            prev = chunks[i - 1].page_content
            # Get the last N characters of the previous chunk
            overlap = prev[-overlap_characters:]
            # Prepend to current chunk
            curr.page_content = overlap + curr.page_content
        overlapped_chunks.append(curr)

    return overlapped_chunks

# Apply overlap (adjust 200 to suit your context)
documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])
